## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-07-10-49-31 +0000,bbc,Healey says more military sites could house as...,https://www.bbc.com/news/articles/c5yvxergw4xo...
1,2025-09-07-10-48-17 +0000,nypost,Off-duty Alaska Airlines pilot who tried shutt...,https://nypost.com/2025/09/07/us-news/off-duty...
2,2025-09-07-10-31-00 +0000,bbc,Ukraine's main government building in Kyiv hit...,https://www.bbc.com/news/articles/cpq5dl5y4nlo...
3,2025-09-07-10-30-16 +0000,wapo,Russia’s largest ever drone attack on Ukraine ...,https://www.washingtonpost.com/world/2025/09/0...
4,2025-09-07-10-24-10 +0000,nyt,"‘If I Live to 25, I’ve Lived a Good Life’",https://www.nytimes.com/2025/09/07/us/wildfire...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2440/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
109,trump,19
119,raid,10
190,immigration,9
19,ukraine,9
100,two,7


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
69,2025-09-07-03-00-00 +0000,wsj,When President Trump unveiled his EU trade dea...,https://www.wsj.com/economy/trade/the-eus-trad...,65
56,2025-09-07-04-24-23 +0000,nypost,South Korean President vows support to over 30...,https://nypost.com/2025/09/07/world-news/south...,60
90,2025-09-07-00-03-18 +0000,nyt,Immigration Raid on Hyundai-LG Plant in Georgi...,https://www.nytimes.com/2025/09/06/world/asia/...,49
158,2025-09-06-15-27-41 +0000,nyt,South Koreans Are Swept Up in Immigration Raid...,https://www.nytimes.com/2025/09/05/us/georgia-...,49
84,2025-09-07-01-00-00 +0000,wsj,"Trump has so far stood by RFK Jr., the health ...",https://www.wsj.com/politics/policy/trump-said...,48


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
69,65,2025-09-07-03-00-00 +0000,wsj,When President Trump unveiled his EU trade dea...,https://www.wsj.com/economy/trade/the-eus-trad...
56,53,2025-09-07-04-24-23 +0000,nypost,South Korean President vows support to over 30...,https://nypost.com/2025/09/07/world-news/south...
17,45,2025-09-07-09-34-43 +0000,cbc,Russia's largest air attack on Ukraine sets fi...,https://www.cbc.ca/news/world/russia-mass-assa...
36,35,2025-09-07-08-42-01 +0000,nypost,Pope Leo XIV names first two saints of his pon...,https://nypost.com/2025/09/07/world-news/pope-...
1,33,2025-09-07-10-48-17 +0000,nypost,Off-duty Alaska Airlines pilot who tried shutt...,https://nypost.com/2025/09/07/us-news/off-duty...
22,32,2025-09-07-09-23-31 +0000,nyt,Prime Minister Shigeru Ishiba of Japan Says He...,https://www.nytimes.com/2025/09/07/world/asia/...
117,31,2025-09-06-20-59-15 +0000,nypost,"Israel issues evacuation orders, tells Palesti...",https://nypost.com/2025/09/06/world-news/israe...
83,26,2025-09-07-01-18-32 +0000,bbc,Friends remember US instructor killed in Lisbo...,https://www.bbc.com/news/articles/cly12lrlxxvo...
103,24,2025-09-06-22-46-40 +0000,nypost,Cheerleader Laken Snelling’s ex-boyfriend prov...,https://nypost.com/2025/09/06/us-news/universi...
84,23,2025-09-07-01-00-00 +0000,wsj,"Trump has so far stood by RFK Jr., the health ...",https://www.wsj.com/politics/policy/trump-said...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
